In [68]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from predict import *
from sklearn.metrics import *
import tensorflow as tf
import warnings
from sklearn.preprocessing import OneHotEncoder 
warnings.filterwarnings('ignore')

In [3]:
PATH_INPUT = os.getcwd() + '/inputs/'

### Load do dataset de treino e separacao dos dados

Load do cvs com o treino feito anteriormente

In [145]:
df_treino = pd.read_csv(PATH_INPUT+'training_set_2.csv', usecols=['name','isMarried', 'isNoble','numDeadRelations','popularity','isAlive'])

Carrega o X e y

In [146]:
X = df_treino.iloc[:,1:5]
y = df_treino.iloc[:,5:]

Split do dataframe em treino e teste

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
X_validacao, X_test, y_validacao, y_test = train_test_split(X_test, y_test, test_size=0.2, shuffle=True) 

Normaliza o X

In [148]:
std_scale = preprocessing.MinMaxScaler()

X_train['numDeadRelations'] = std_scale.fit_transform((X_train['numDeadRelations'].values).reshape((-1,1)))
X_train['popularity'] = std_scale.fit_transform((X_train['popularity'].values).reshape((-1,1)))

In [149]:
X_validacao['numDeadRelations'] = std_scale.fit_transform((X_validacao['numDeadRelations'].values).reshape((-1,1)))
X_validacao['popularity'] = std_scale.fit_transform((X_validacao['popularity'].values).reshape((-1,1)))

In [150]:
X_test['numDeadRelations'] = std_scale.fit_transform((X_test['numDeadRelations'].values).reshape((-1,1)))
X_test['popularity'] = std_scale.fit_transform((X_test['popularity'].values).reshape((-1,1)))

In [151]:
X_train,X_validacao,X_test,y_train,y_validacao,y_test = X_train.values,X_validacao.values,X_test.values,y_train.values,y_validacao.values,y_test.values

In [152]:
# Begin building the model framework
# Declare the variables that need to be learned and initialization
# There are 4 features here, A's dimension is (4, 1)
A = tf.Variable(tf.random_normal(shape=[4, 1]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [153]:
# Define placeholders
data = tf.placeholder(dtype=tf.float32, shape=[None, 4])
target = tf.placeholder(dtype=tf.float32, shape=[None, 1])

In [154]:
# Declare the model you need to learn
mod = tf.matmul(data, A) + b

In [155]:
# Declare loss function
# Use the sigmoid cross-entropy loss function,
# first doing a sigmoid on the model result and then using the cross-entropy loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels=target))

In [186]:
# Define the learning rate， batch_size etc.
learning_rate = 0.0001
batch_size = 5
iter_num = 15000

In [187]:
# Define the optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate)

In [188]:
# Define the goal
goal = opt.minimize(loss)

In [189]:
# Define the accuracy
# The default threshold is 0.5, rounded off directly
prediction = tf.round(tf.sigmoid(mod))
# Bool into float32 type
correct = tf.cast(tf.equal(prediction, target), dtype=tf.float32)
# Average
accuracy = tf.reduce_mean(correct)
# End of the definition of the model framework

In [190]:
# Start training model
# Define the variable that stores the result
loss_trace = []
train_acc = []
test_acc = []

In [191]:
# training model
for epoch in range(iter_num):
    # Generate random batch index
    batch_index = np.random.choice(len(X_train), size=batch_size)
    batch_train_X = X_train[batch_index]
    batch_train_y = np.matrix(y_train[batch_index])
    sess.run(goal, feed_dict={data: batch_train_X, target: batch_train_y})
    temp_loss = sess.run(loss, feed_dict={data: batch_train_X, target: batch_train_y})

    # convert into a matrix, and the shape of the placeholder to correspond
    temp_train_acc = sess.run(accuracy, feed_dict={data: X_train, target: np.matrix(y_train)})
    temp_test_acc = sess.run(accuracy, feed_dict={data: X_validacao, target: np.matrix(y_validacao)})
    
    # recode the result
    loss_trace.append(temp_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
    # output
    if (epoch + 1) % 300 == 0:
        print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss, temp_train_acc, temp_test_acc))

epoch:  300 loss: 0.985438 train_acc: 0.735861 test_acc: 0.778846
epoch:  600 loss: 0.544777 train_acc: 0.735861 test_acc: 0.778846
epoch:  900 loss: 0.678129 train_acc: 0.735861 test_acc: 0.778846
epoch: 1200 loss: 0.516371 train_acc: 0.735861 test_acc: 0.778846
epoch: 1500 loss: 0.949077 train_acc: 0.735861 test_acc: 0.778846
epoch: 1800 loss: 0.526411 train_acc: 0.735861 test_acc: 0.778846
epoch: 2100 loss: 1.079272 train_acc: 0.735861 test_acc: 0.778846
epoch: 2400 loss: 0.450789 train_acc: 0.735861 test_acc: 0.778846
epoch: 2700 loss: 0.533101 train_acc: 0.735861 test_acc: 0.778846
epoch: 3000 loss: 0.318492 train_acc: 0.735861 test_acc: 0.778846
epoch: 3300 loss: 0.337722 train_acc: 0.735861 test_acc: 0.778846
epoch: 3600 loss: 0.309120 train_acc: 0.735861 test_acc: 0.778846
epoch: 3900 loss: 0.830063 train_acc: 0.735861 test_acc: 0.778846
epoch: 4200 loss: 0.678227 train_acc: 0.735861 test_acc: 0.778846
epoch: 4500 loss: 0.323200 train_acc: 0.735861 test_acc: 0.778846
epoch: 480

In [192]:
theta = sess.run(A)
theta

array([[-0.08577682],
       [-0.41309524],
       [ 0.79364973],
       [ 0.9717    ]], dtype=float32)

In [193]:
bias = sess.run(b)
bias

array([[1.1283875]], dtype=float32)

In [194]:
y_prob = lambda x_pred: x_pred.dot(theta) + bias

In [195]:
y_pred = sess.run(tf.sigmoid(y_prob(X_validacao)))
y_pred = 1 * (y_pred >= 0.5)

In [196]:
confusion_matrix(y_validacao, y_pred)

array([[  0,  69],
       [  0, 243]])

In [197]:
accuracy_score(y_validacao, y_pred)

0.7788461538461539

Dataset de teste

In [198]:
y_pred = sess.run(tf.sigmoid(y_prob(X_test)))
y_pred = 1 * (y_pred >= 0.5)

In [199]:
confusion_matrix(y_test, y_pred)

array([[ 0, 15],
       [ 0, 63]])

In [200]:
accuracy_score(y_test, y_pred)

0.8076923076923077